<a href="https://colab.research.google.com/github/Nithinps021/Project_IISU/blob/main/botfirsttrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
! pip3 install nltk

In [8]:
! pip3 install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [14]:
#to import the data
with open('intents.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"patterns":inputs,
                     "tags":tags})
print(data)

                                             patterns                   tags
0                                            Hi there               greeting
1                                         How are you               greeting
2                                    Is anyone there?               greeting
3                                                 Hey               greeting
4                                                Hola               greeting
5                                               Hello               greeting
6                                            Good day               greeting
7                                                 Bye                goodbye
8                                       See you later                goodbye
9                                             Goodbye                goodbye
10                          Nice chatting to you, bye                goodbye
11                                     Till next time                goodbye

In [13]:
print(data)

                                               inputs                   tags
0                                            Hi there               greeting
1                                         How are you               greeting
2                                    Is anyone there?               greeting
3                                                 Hey               greeting
4                                                Hola               greeting
5                                               Hello               greeting
6                                            Good day               greeting
7                                                 Bye                goodbye
8                                       See you later                goodbye
9                                             Goodbye                goodbye
10                          Nice chatting to you, bye                goodbye
11                                     Till next time                goodbye

In [17]:
#removing punctuations
import string
data['patterns'] = data['patterns'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['patterns'] = data['patterns'].apply(lambda wrd: ''.join(wrd))

In [18]:
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])
train = tokenizer.texts_to_sequences(data['patterns'])

In [19]:
#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)


In [20]:
#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [21]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

9
number of unique words :  89
output length:  9


In [22]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
2/2 [==============================] - 2s 11ms/step - loss: 2.1960 - accuracy: 0.2128
Epoch 2/200
2/2 [==============================] - 0s 11ms/step - loss: 2.1938 - accuracy: 0.2553
Epoch 3/200
2/2 [==============================] - 0s 9ms/step - loss: 2.1922 - accuracy: 0.2553
Epoch 4/200
2/2 [==============================] - 0s 10ms/step - loss: 2.1903 - accuracy: 0.2128
Epoch 5/200
2/2 [==============================] - 0s 10ms/step - loss: 2.1884 - accuracy: 0.2979
Epoch 6/200
2/2 [==============================] - 0s 15ms/step - loss: 2.1866 - accuracy: 0.3404
Epoch 7/200
2/2 [==============================] - 0s 11ms/step - loss: 2.1847 - accuracy: 0.2553
Epoch 8/200
2/2 [==============================] - 0s 11ms/step - loss: 2.1828 - accuracy: 0.2553
Epoch 9/200
2/2 [==============================] - 0s 10ms/step - loss: 2.1807 - accuracy: 0.2128
Epoch 10/200
2/2 [==============================] - 0s 11ms/step - loss: 2.1787 - accuracy: 0.2128
Epoch 11/200
2/2 [==

In [24]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Going Merry : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : give me blood result
Going Merry :  My pleasure
You : open adverse drugs module
Going Merry :  Navigating to Adverse drug reaction module
You : /stop
Going Merry :  Good to see you again


KeyboardInterrupt: ignored